In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from vgg_models.vgg import vgg13_bn

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Load the CIFAR-10 test dataset -------------

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

#--------- Load the permuted dataset ----------

class CustomDataset(Dataset):
    def __init__(self, dataset_dict, transform = None):
        self.dataset_dict = dataset_dict
        self.transform = transform

    def __len__(self):
        return len(self.dataset_dict)

    def __getitem__(self, index):
        data = self.dataset_dict[index]
        image = data['image']
        if self.transform:
            image = self.transform(data['image'])
        label = data['label']

        return image, label

# Load the dataset dictionary
loaded_dataset = torch.load('cifar10_permuted.pth')

# Create an instance of the CustomDataset
permuted_dataset = CustomDataset(loaded_dataset, transform = transform)
permuted_loader = DataLoader(permuted_dataset, batch_size=1024, shuffle=False)

# ------------- Load the pretrained model ------------
pretrained_model = vgg13_bn(pretrained=True)
pretrained_model = pretrained_model.to(device)
pretrained_model.eval()


# -----------calculate accuracies -----------
correct_predictions = 0
total_samples = 0

correct_predictions_permuted = 0
total_samples_permuted = 0

with torch.no_grad():
    for inputs, labels in test_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = pretrained_model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Update counts
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    for inputs, labels in permuted_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = pretrained_model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Update counts
        total_samples_permuted += labels.size(0)
        correct_predictions_permuted += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f'Accuracy on CIFAR-10: {100 * accuracy:.2f}%')

# Calculate accuracy on permuted dataset
accuracy_permuted = correct_predictions_permuted / total_samples_permuted
print(f'Accuracy on permuted CIFAR-10: {100 * accuracy_permuted:.2f}%')


Files already downloaded and verified
Accuracy on CIFAR-10: 94.21%
Accuracy on permutrd CIFAR-10: 32.58%
